In [ ]:
!pip install mediapipe
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is inco

In [ ]:
# base
import pandas as pd
import numpy as np
import os
from zipfile import ZipFile
from tqdm import tqdm
import uuid
pd.set_option('display.max_colwidth', None)


# base ML
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# NN
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn as nn

# models
import cv2
from google.colab.patches import cv2_imshow
import mediapipe as mp
from ultralytics import YOLO
from torchvision.models.segmentation import deeplabv3_resnet50

# visualization
import plotly.express as px
import plotly.graph_objs as go

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'); device # переходим на гпу

device(type='cpu')

In [ ]:
# размеченные видео по классам элементов
ELEMENTS_PATH = r'/content/drive/MyDrive/диплом/По элементам'
# ссылка на датасет zip файла
ZIP_PATH = '/content/drive/MyDrive/диплом/Datasets/Элементы.zip'
# папка для сохранения аугментированных видео для редких классов
AUG_PATH =r'/content/drive/MyDrive/диплом/Аугментация'
# директория со всеми промежуточными датасетами
DATASETS_PATH = '/content/drive/MyDrive/диплом/Datasets'
# директория, где сохранены преобразованные исходные видео в силуэты с pose estimation
SILHOUETTE_PATH = '/content/drive/MyDrive/диплом/Video silhouettes new'

# Load video data

> Создаем из директории с элементами zip-файл, из которого будет состоять датасет

In [ ]:
# создаем zip-файл со всем элементами для датасета
# !zip -r '/content/drive/MyDrive/диплом/Datasets/Элементы.zip' '/content/drive/MyDrive/диплом/По элементам'
# !zip -r ZIP_PATH ELEMENTS_PATH

In [ ]:
# zipfile = ZipFile(ZIP_PATH, "r")
# zipfile.extractall('Элементы.zip')
# zipfile.printdir()

In [ ]:
# датасет: путь до видео, лейбл (название элемента)

df_elements = pd.DataFrame()
BD_path, BD_names = [],[]
for group in os.listdir(ELEMENTS_PATH):  # равновесия, прыжки, повороты
  df = pd.DataFrame()
  for elem_name in os.listdir(os.path.join(ELEMENTS_PATH,group)):  # название классов, лейблы
    for video_name in os.listdir(os.path.join(ELEMENTS_PATH,group,elem_name)):  # название видео
      BD_path.append(os.path.join(ELEMENTS_PATH,group,elem_name, video_name))
      BD_names.append(f'{group}_{elem_name}') # название класса

df_elements['BD_path'] = BD_path
df_elements['BD_names'] = BD_names
print(df_elements.shape)
df_elements.head(3)

(3311, 2)


,BD_path,BD_names
0,/content/drive/MyDrive/диплом/По элементам/Повороты/Пассе/поворот пассе — сделано в Clipchamp.mp4,Повороты_Пассе
1,/content/drive/MyDrive/диплом/По элементам/Повороты/Пассе/Диана Чугунихина поворт пассе — сделано в Clipchamp.mp4,Повороты_Пассе
2,/content/drive/MyDrive/диплом/По элементам/Повороты/Пассе/Диана Чугунихина поворт пассе — сделано в Clipchamp (1).mp4,Повороты_Пассе


In [ ]:
# количество уникальных видео в датасете и уникальных классов
df_elements['BD_path'].nunique(), df_elements['BD_names'].nunique()

(3311, 80)

Всего собрано 3311 видео фрагментов элементов, Классов элементов -- 80

In [ ]:
# # Преобразуем классы в индексы, присвоив порядковый номер класса через LabelEncoder

le = LabelEncoder()
df_elements['BD_label'] = le.fit_transform( df_elements['BD_names'])
df_elements.head(3)

,BD_path,BD_names,BD_label
0,/content/drive/MyDrive/диплом/По элементам/Повороты/Пассе/поворот пассе — сделано в Clipchamp.mp4,Повороты_Пассе,15
1,/content/drive/MyDrive/диплом/По элементам/Повороты/Пассе/Диана Чугунихина поворт пассе — сделано в Clipchamp.mp4,Повороты_Пассе,15
2,/content/drive/MyDrive/диплом/По элементам/Повороты/Пассе/Диана Чугунихина поворт пассе — сделано в Clipchamp (1).mp4,Повороты_Пассе,15


In [ ]:
# Группируем и считаем количество видео на класс
df_counts = df_elements.groupby('BD_label')['BD_path'].count().reset_index()
df_counts.columns = ['BD_label', 'BD_count']

# Строим график
fig = px.bar(df_counts, x='BD_label', y='BD_count',
             title='Распределение видео по классам',
             labels={'BD_label': 'Класс', 'BD_count': 'Количество видео'},
             text='BD_count')


fig.update_layout(plot_bgcolor='black',
                   paper_bgcolor='black',
                   font_color='white')

fig.update_layout(xaxis_tickangle=-45)
fig.show()

## Augmentation

Для увеличения размера трейнового датасета и выравнивания дисбаланса классов проведем аугментацию

In [ ]:
df_elements['BD_label'].value_counts()

,count
BD_label,
22,355
41,260
8,258
20,164
23,130
...,...
31,2
59,1
32,1


In [ ]:
np.median(df_elements['BD_label'].value_counts().values)  # медиана количества раз появления класса в датасете = 16.5

16.5

In [ ]:
# топ самых редких
df_elements['BD_count'] = df_elements.groupby('BD_label')['BD_path'].transform('count')
df_elements.sort_values('BD_count', ascending = True).head(3)

,BD_path,BD_names,BD_label,BD_count
2569,/content/drive/MyDrive/диплом/По элементам/Прыжки/«Фуете» с подбивом/Самирахон_Салиева касаясь с отворотом — сделано в Clipchamp.mp4,Прыжки_«Фуете» с подбивом,32,1
2018,/content/drive/MyDrive/диплом/По элементам/Равновесия/Арабеск с туловищем вперед в горизонтальном положении/Дарья_Тараканова равновесие крокодил — сделано в Clipchamp.mp4,Равновесия_Арабеск с туловищем вперед в горизонтальном положении,59,1
1316,/content/drive/MyDrive/диплом/По элементам/Повороты/В шпагате с помощью с наклоном вперед/колбаса с наклоном вперед — сделано в Clipchamp.mp4,Повороты_В шпагате с помощью с наклоном вперед,5,1


> Редкими классами будут классы с элементами, которые встречаются в выборке реже 20 раз. Для их увеличения будут применяться методы аугментации

In [ ]:
# если элемент встречается реже 20 раз, делаем аугментацию для редких классов
df_elements['augmentation'] = np.where(df_elements.groupby('BD_label')['BD_path'].transform('count') < 20, True, False)
df_elements['augmentation'] .value_counts()

,count
augmentation,
False,2982
True,329


In [ ]:
# # количество видео, которые будут аугментированы (некорректный график)
# platforms = df_elements['augmentation'].value_counts()

# fig1 = px.pie(names=platforms.index,
#                values=platforms.values,
#                title='Количество видео для аугментации',
#                color_discrete_sequence=['lime'])

# fig1.update_layout(plot_bgcolor='black',
#                    paper_bgcolor='black',
#                    font_color='white')
# fig1.update_layout(height=400, width=500)

# fig1.show()

In [ ]:
# количество классов, которые будут аугментированы и нет
df_elements[df_elements['augmentation'] ]['BD_label'].nunique(), df_elements[~df_elements['augmentation'] ]['BD_label'].nunique()

(43, 37)

In [ ]:
# Вычисляем количество уникальных классов для аугментированных и неаугментированных данных
augmented_count = df_elements[df_elements['augmentation']]['BD_label'].nunique()
non_augmented_count = df_elements[~df_elements['augmentation']]['BD_label'].nunique()

# Данные для pie chart
labels = ['Классы с количеством элементов < 20', 'Классы с количеством элементов >= 20']
values = [augmented_count, non_augmented_count]

# Создаем pie chart
fig = px.pie(names=labels, values=values,
                             title='Соотношение аугментированных и неаугментированных классов',
                             color_discrete_sequence=['lime']
                            )


fig.update_layout(plot_bgcolor='black',
                   paper_bgcolor='black',
                   font_color='white')
fig.update_layout(height=500, width=800)

fig.show()

Для 43 классов будут применены методы аугментации

Так как имеется сильный дисбаланс классов, будем применять методы аугментации для редких классов:
- Сдвиг (по оси Ox и Oy, влево/вправо и вверх/вниз)
- Масштабирование (небольшое изменение расстояния до камеры)

In [ ]:
# Сдвиг, небольшое смещение силуэта в рамках кадра (влево/вправо/вверх/вниз)
def translate_frame(frame, tx, ty):  # frame - кадр (изображение), max_shift - максимальный сдвиг по пикселям
  h, w, _ = frame.shape  # высота, ширина, число каналов (3: RGB)
  # tx = np.random.randint(-max_shift, max_shift)  # случайное значение, на которое будет сдвиг по горизонтали по оси Ox влево/вправо
  # ty = np.random.randint(-max_shift, max_shift)  # случайное значение, на которое будет сдвиг по вертикали по оси Oy вверх/вниз
  M = np.float32([[1, 0, tx], [0, 1, ty]]) # матрица трансформации для cv2.warpAffine
  return cv2.warpAffine(frame, M, (w, h), borderValue=(0, 0, 0))  # матрица аффинного преобразования, применяет преобразование к кадру
                                          # Пустые области (после сдвига) заливаются чёрным цветом (0, 0, 0)

In [ ]:
# scaling, масштабирование, приближение/отдаление кадра
def scaling_frame(frame, scale):   #  кадр может быть уменьшен до 70% или увеличен до 110%
    h, w, _ = frame.shape   # высота, ширина кадра, число каналов (3: rgb)
    # scale = np.random.uniform(*scale_range)  # случайный коэффициент масштабирования
    resized = cv2.resize(frame, (int(w * scale), int(h * scale)))  #  изменяем размер кадра

    # Вписываем обратно в исходный размер
    padded = np.zeros_like(frame)   # создаём чёрное изображение того же размера
    min_h = min(h, resized.shape[0])
    min_w = min(w, resized.shape[1])
    padded[:min_h, :min_w] = resized[:min_h, :min_w]   # изменяем размер кадра, вставляем масштабированное изображение в чёрный фон
    return padded

In [ ]:
# аугментация редких классов
# читаем оригинальное видео
# добавляем сдвиг и масштабирование
# сохраняем новое видео
def read_aug_save_video(input_video_path, remaining):

  cap = cv2.VideoCapture(input_video_path)  # открываем видео

  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # ширина кадра
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # высота кадра
  fps = cap.get(cv2.CAP_PROP_FPS)  # частота кадров
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')

  output_video_path = input_video_path.split('/')[-1][:-4] + f' aug_{remaining}.mp4'  # название видер для сохранения
  out = cv2.VideoWriter(os.path.join(AUG_PATH,output_video_path), fourcc, fps, (width, height))  # замедляем видео в 4 раза

  tx = np.random.randint(-100, 100)  # случайное значение, на которое будет сдвиг по горизонтали по оси Ox влево/вправо
  ty = np.random.randint(-100, 100)  # случайное значение, на которое будет сдвиг по вертикали по оси Oy вверх/вниз
  scale = np.random.uniform(*(0.7, 1.1))  # случайный коэффициент масштабирования

  while cap.isOpened():  # если видео открылось
    ret, frame = cap.read()  # читаем кадр
    if not ret:  # если кадры закончились
        break
    frame = translate_frame(frame, tx,ty)  # сдвиг
    frame = scaling_frame(frame, scale)    # масштабирование кадра

    out.write(frame)  # сохраняем новое видео
    # cv2_imshow(frame)

  cap.release()
  out.release()


Алгоритм аугментации видео для редких классов:

1. Группировка по классам: выбираются только классы, где количество видео < 20.
2. Расчёт недостающих видео: для каждого класса вычисляется, сколько видео нужно добавить до 20.
3. Равномерное распределение аугментаций: недостающие видео равномерно распределяются между уже существующими (остаток — первым в списке).
4. Аугментация и сохранение: для каждого оригинального видео создаётся нужное количество аугментированных копий и сохраняются в отдельную папку.
5. Формирование итогового датафрейма df_aug: в него добавляются пути, имена и метки новых видео.

In [ ]:
df_aug = pd.DataFrame(columns=['BD_path', 'BD_names', 'BD_label'])  # датасет с новыми видео

rare_classes = df_elements[df_elements['augmentation']].groupby('BD_label')  # редкие классы
for BD_label, group in tqdm(rare_classes, desc="Аугментация редких классов"):
    current_count = len(group)
    print(f"Класс: {BD_label}, текущих видео: {current_count}")

    if current_count >= 20:
        continue  # Пропускаем классы, где уже >= 20

    # Сколько всего нужно добавить до 20
    to_add_total = 20 - current_count
    videos = group[['BD_path', 'BD_names']].values

    # Сколько раз аугментировать каждое видео
    repeats = [to_add_total // current_count] * current_count
    for i in range(to_add_total % current_count):
        repeats[i] += 1  # остаток распределяем по первым видео

    # Генерация новых примеров
    for (BD_path, BD_name), n_aug in zip(videos, repeats):
        for i in range(n_aug):
            unique_id = uuid.uuid4().hex[:8]   # уникальный путь, чтобы не было дублей
            aug_name = f"{BD_name}_aug_{unique_id}"
            SAVE_PATH = os.path.join(AUG_PATH, BD_path.split('/')[-1][:-4] + f' aug_{i}.mp4')
            # print(SAVE_PATH)
            read_aug_save_video(BD_path, i)  # аугментируем и сохраняем

            new_row = pd.DataFrame({
                'BD_path': [SAVE_PATH],
                'BD_names': [aug_name],
                'BD_label': [BD_label]
            })
            df_aug = pd.concat([df_aug, new_row], ignore_index=True)
            # print(f"Добавлено видео: {SAVE_PATH}, Обновлённый размер df_aug: {df_aug.shape}")

print(df_aug.shape)
df_aug.head()

Аугментация редких классов:   0%|          | 0/43 [00:00<?, ?it/s]

Класс: 0, текущих видео: 17


Аугментация редких классов:   2%|▏         | 1/43 [00:08<05:52,  8.40s/it]

Класс: 4, текущих видео: 13


Аугментация редких классов:   5%|▍         | 2/43 [00:31<11:39, 17.05s/it]

...

Аугментация редких классов:  98%|█████████▊| 42/43 [27:58<00:40, 40.02s/it]

Класс: 78, текущих видео: 3


Аугментация редких классов: 100%|██████████| 43/43 [28:44<00:00, 40.11s/it]

(531, 3)


,BD_path,BD_names,BD_label
0,/content/drive/MyDrive/диплом/Аугментация/Саша_Семенова поворот назад арабеск — сделано в Clipchamp aug_0.mp4,Повороты_Арабеск_aug_295c689e,0
1,/content/drive/MyDrive/диплом/Аугментация/Саша_Семенова поворот назад арабеск — сделано в Clipchamp (1) aug_0.mp4,Повороты_Арабеск_aug_34da5187,0
2,/content/drive/MyDrive/диплом/Аугментация/Саша_Семенова поворот назад арабеск — сделано в Clipchamp (2) aug_0.mp4,Повороты_Арабеск_aug_484d50dd,0
3,/content/drive/MyDrive/диплом/Аугментация/поворот планше с рукой — сделано в Clipchamp aug_0.mp4,"Повороты_Боковой шпагат с помощью, туловище в горизонтальном положении_aug_b9dd3ab0",4
4,/content/drive/MyDrive/диплом/Аугментация/Стиляна_Николова поворот планше с рукой обратный — сделано в Clipchamp aug_0.mp4,"Повороты_Боковой шпагат с помощью, туловище в горизонтальном положении_aug_3417af72",4


In [ ]:
# объединяем в общий датасет исходные и аугментированные данные
df_elements_new = pd.concat([df_elements[['BD_path', 'BD_names','BD_label']], df_aug], axis = 0)
df_elements_new.shape

(3842, 3)

Также для увеличения размера трейнового датасета для всех видео будет применен еще один метод аугментации -- горизонтальное отражение (отзеркаливание) видео, поскольку элементы в художественной гимнастике могут выполняться в обе стороны (как на левую, так и на правую ногу)

In [ ]:
# проставим флаг Flip для тех видео, которые потом будут отзеркалены по горизонтали
# умножим датасет на 2 и вторую половину будем отзеркаливать

df_elements_new['flip'] = False
df_elements_flip = df_elements_new.copy()
df_elements_flip['flip'] = True
df_elements_new = pd.concat([df_elements_new, df_elements_flip], axis = 0)
df_elements_new.reset_index(drop = True, inplace = True)
print(df_elements_new.shape)  # размерность датасета увеличилась в 2 раза
df_elements_new.flip.value_counts()

(7684, 4)


,count
flip,
False,3842
True,3842


> Далее удалим из датасета часть элементов, которые встречаются слишком часто по сравнению с другими классами.

In [ ]:
# Группируем и считаем количество видео на класс
df_counts = df_elements_new.groupby('BD_label')['BD_path'].count().reset_index()
df_counts.columns = ['BD_label', 'BD_count']

# Строим график
fig = px.bar(df_counts, x='BD_label', y='BD_count',
             title='Распределение видео по классам',
             labels={'BD_label': 'Класс', 'BD_count': 'Количество видео'},
             text='BD_count')


fig.update_layout(plot_bgcolor='black',
                   paper_bgcolor='black',
                   font_color='white')

fig.update_layout(xaxis_tickangle=-45)
fig.show()

По графику распределения видно:

1. Сильный дисбаланс классов все еще присутствует — некоторые классы имеют до 600-700 видео, в то время как другие — менее 100.

3. Много классов с малым числом видео — что может привести к перекосу в обучении модели: она будет переобучаться на часто встречающиеся классы и игнорировать редкие.

Для популярных классов оставим не более 200 рандомных видео в каждом классе, чтобы модель не переобучалась на частовстречающихся классах

In [ ]:
# пересчитываем количество
df_elements_new['BD_count'] = df_elements_new.groupby('BD_label')['BD_path'].transform('count')
df_elements_new[df_elements_new['BD_count'] > 200]['BD_label'].nunique()  # у 8 классов количество видео превышает 200, оставим только 200 рандомных видео

8

In [ ]:
# популярные классы
popular_classes = df_elements_new[df_elements_new['BD_count'] > 200]['BD_label'].unique()
popular_classes

array([8, 22, 21, 23, 20, 66, 38, 41], dtype=object)

In [ ]:
# для популярных классов (>200), оставим только 200 рандомных
save_indexes = []
for pop_class in popular_classes:
  save_indexes.append(np.random.choice(
      df_elements_new[
      df_elements_new['BD_label'] == pop_class
    ].index,
        size = 200, replace = False  # 200 рандомных видео, без повторений
    ).tolist())
print(len(np.unique(save_indexes)) ) # 8 классов по 200
# добавляем видео, которых <= 200 в классе

save_indexes = sum(save_indexes, df_elements_new[~df_elements_new['BD_label'].isin(popular_classes)].index.tolist())
print(len(np.unique(save_indexes)))

1600
6246


In [ ]:
df_elements = df_elements_new[df_elements_new.index.isin(save_indexes)].reset_index(drop = True)
df_elements.shape

(6246, 5)

In [ ]:
# Группируем и считаем количество видео на класс
df_counts = df_elements.groupby('BD_label')['BD_path'].count().reset_index()
df_counts.columns = ['BD_label', 'BD_count']

fig = px.bar(df_counts, x='BD_label', y='BD_count',
             title='Распределение видео по классам',
             labels={'BD_label': 'Класс', 'BD_count': 'Количество видео'},
             text='BD_count')


fig.update_layout(plot_bgcolor='black',
                   paper_bgcolor='black',
                   font_color='white')

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
# пересчитываем количество элементов в каждом классе и медиану
df_elements['BD_count'] = df_elements.groupby('BD_label')['BD_path'].transform('count')
df_elements['BD_count'].min(), df_elements['BD_count'].max(), df_elements['BD_count'].median(), df_elements['BD_count'].mean()

(40, 200, 112.0, 120.97790585975024)

In [ ]:
df_elements['video_name'] = df_elements['BD_path'].str.split('/').apply(lambda x: x[-1])
df_elements.head(2)

,BD_path,BD_names,BD_label,flip,BD_count,video_name
0,/content/drive/MyDrive/диплом/По элементам/Повороты/Пассе/поворот пассе — сделано в Clipchamp.mp4,Повороты_Пассе,15,False,40,поворот пассе — сделано в Clipchamp.mp4
1,/content/drive/MyDrive/диплом/По элементам/Повороты/Пассе/Диана Чугунихина поворт пассе — сделано в Clipchamp.mp4,Повороты_Пассе,15,False,40,Диана Чугунихина поворт пассе — сделано в Clipchamp.mp4


После всех преобразований минимальное количество классов стало 40, максимальное 200, медиана увеличилась с 11 до 132

In [ ]:
# кол-во видео в разрезе для каждой группы БД

df_elements['BD_group'] = df_elements['BD_names'].str.split('_').apply(lambda x: x[0])
grouped_data = df_elements.groupby('BD_group')['BD_count'].count().reset_index()
display(grouped_data)

# круговая диаграмма
fig = px.pie(grouped_data,
             values='BD_count',
             names='BD_group',
             title='Количество видео по группам БД',
             hover_data=['BD_count'],
              color_discrete_sequence=['orange'])

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(plot_bgcolor='black',
                  paper_bgcolor='black',
                  font_color='white')
fig.show()

,BD_group,BD_count
0,Повороты,2408
1,Прыжки,2052
2,Равновесия,1786


- Выборка стала более сбалансированной. Повороты не преобладают так сильно, как было до преобразований

In [ ]:
# задаем уникальный путь для считывания, если есть дубли и одинаковые названия, куда будут сохраняться преобразованные кадры (силуэты)

df_elements['BD_silhouette_path'] = df_elements['video_name'].apply(lambda x: os.path.join(SILHOUETTE_PATH,x))
df_elements.reset_index(drop = True, inplace = True)
df_elements['BD_silhouette_path'] = (df_elements['BD_silhouette_path'].str.split('.mp4').apply(lambda x: x[0]) +'_'+ df_elements.index.astype(str) + '.mp4').values.tolist()
print(df_elements['BD_silhouette_path'].nunique(), df_elements.shape)
df_elements['BD_silhouette_path'].tail(6)

6246 (6246, 8)


,BD_silhouette_path
6240,/content/drive/MyDrive/диплом/Video silhouettes new/Дарья_Тараканова равновесие вперед 90 — сделано в Clipchamp aug_5_6240.mp4
6241,/content/drive/MyDrive/диплом/Video silhouettes new/Саша_Семенова равновесие вперед 90 — сделано в Clipchamp aug_0_6241.mp4
6242,/content/drive/MyDrive/диплом/Video silhouettes new/Саша_Семенова равновесие вперед 90 — сделано в Clipchamp aug_1_6242.mp4
6243,/content/drive/MyDrive/диплом/Video silhouettes new/Саша_Семенова равновесие вперед 90 — сделано в Clipchamp aug_2_6243.mp4
6244,/content/drive/MyDrive/диплом/Video silhouettes new/Саша_Семенова равновесие вперед 90 — сделано в Clipchamp aug_3_6244.mp4
6245,/content/drive/MyDrive/диплом/Video silhouettes new/Саша_Семенова равновесие вперед 90 — сделано в Clipchamp aug_4_6245.mp4


In [ ]:
# сохраним промежуточный результат
df_elements.to_excel(os.path.join(DATASETS_PATH,'Аугментированный_датасет.xlsx'), index = False)

In [ ]:
df_elements.shape

(6246, 8)

## Dataset

Для формирования датасета перед обучением:

1. На вход подается оригинальное видео
2. С помощью YOLOv8 выполняется детекция человека на видео, выбирается передний план (самая обширная область)
3. Сегментация человека на видео с помощью DeepLab внутри выделенного бокса
2. Выполняется оценка позы (pose estiamtion) с помощью media pipe, определяются 32 точки, соединяющие части тела
3. Линии для каждой части тела рисуются отдельным цветом, чтобы ноги, руки и туловище были более отличимы
4. Удаление фона, на выходе остаются только сегментированный силуэт человека, точки (суставы) и линии, соединяющие суствы


In [ ]:
df_elements = pd.read_excel(os.path.join(DATASETS_PATH,'Аугментированный_датасет.xlsx'))
df_elements.shape, df_elements['BD_silhouette_path'].nunique()

((6246, 8), 6246)

In [ ]:
df_elements_orig = df_elements[~df_elements['flip']]  # оставим только оригинальные видео, обработаем (детекция, сегментация, оценка позы,отрисовка)
# отзеркалим уже полученные после обработки видео для экономии времени
df_elements_orig.shape

(3135, 8)

In [ ]:
# Модели

yolo_model = YOLO('yolov8s.pt') # загружаем веса предобученной модели
deeplab_model = deeplabv3_resnet50(pretrained=True).to(device).eval()  # сегментация человека, предобученная резнет50

# MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    model_complexity=2,  # более точная сложность модели
    static_image_mode=False,  # обработка видеопотока, связывает порядок кадров
    min_detection_confidence=0.25,  # минимальный порог уверенности, что на видео есть человек
    min_tracking_confidence=0.4  # минимальный порог точек
)

# Нормализация для DeepLab
transform = T.Compose([
    T.ToPILImage(),   # перевод в изображение
    T.Resize((520, 520)),
    T.ToTensor(),  # в тензор
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])   # нормализация цветов
])

# Цвета для отрисовки частей тела
COLORS = {
    'left_arm': (0, 255, 0),  # зеленый
    'right_arm': (0, 200, 0),
    'left_leg': (0, 0, 255),  # красный
    'right_leg': (0, 0, 200),
    'torso': (255, 0, 0),   # синий
    'angle_lines': (255, 255, 0)
}

# Части тела
LEFT_ARM = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST]  # левая рука
RIGHT_ARM = [mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST]  # правая рука
LEFT_LEG = [mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE]  # левая нога
RIGHT_LEG = [mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE] # правая нога
TORSO = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
         mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.LEFT_HIP]   # туловище

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# функция обработки видео для подачи в модель
def processing_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)   # открываем видео
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # ширина
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))   # высота

    fps = cap.get(cv2.CAP_PROP_FPS)  # частота кадров в 1 секунде

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Основной цикл обработки видео
    while cap.isOpened():  # открываем видео
        ret, frame = cap.read()  # читаем кадры
        if not ret:  # кадры закончились
            break

        # преобразование в ргб
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 1. Детекция человека с помощью YOLO
        results = yolo_model.predict(frame, classes=[0], verbose=False)  # детекция человека,поиск боксов с людьми
        if results and results[0].boxes is not None and len(results[0].boxes) > 0:  # если люди нашлись
            boxes = results[0].boxes.xyxy.cpu().numpy()

            # Выбираем человека на переднем плане по максимальной нижней координате (самый ближний), плохо для прыжков
            bottoms = [y2 for x1, y1, x2, y2 in boxes]
            main_box = boxes[np.argmax(bottoms)]
            x1, y1, x2, y2 = map(int, main_box)
        else:
            x1, y1, x2, y2 = 0, 0, width, height

        # 2. Сегментация DeepLab внутри выбранного бокса
        cropped = frame_rgb[y1:y2, x1:x2]  # детекция на детектированную yolo область
        if cropped.size == 0:  # если нет области детекции (человек не найден), закрашиваем черным
            out.write(np.zeros_like(frame))
            continue
        # сегментируем силуэт человека и заливаем серой заливкой
        input_tensor = transform(cropped).unsqueeze(0).to(device)
        with torch.no_grad():
            output = deeplab_model(input_tensor)['out'][0]
        output_predictions = output.argmax(0).byte().cpu().numpy()

        person_mask = (output_predictions == 15).astype(np.uint8) * 255  # предикт человека, класс человека = 15
        person_mask = cv2.resize(person_mask, (x2 - x1, y2 - y1), interpolation=cv2.INTER_NEAREST)  # накладываем маску на ближайшего

        full_mask = np.zeros((height, width), dtype=np.uint8)
        full_mask[y1:y2, x1:x2] = person_mask

        # 3. Наложение силуэта
        black_frame = np.zeros_like(frame)  # черный фон
        silhouette_color = (180, 180, 180)  # серый силуэт
        silhouette = black_frame.copy()
        silhouette[full_mask == 255] = silhouette_color

        # 4. MediaPipe: суставы и линии на оригинальном кадре
        results_pose = pose.process(frame_rgb)
        if results_pose.pose_landmarks:  # определяем точки суставов
            landmarks = results_pose.pose_landmarks.landmark
            h, w, _ = frame.shape

            def get_point(lm):
                return int(landmarks[lm].x * w), int(landmarks[lm].y * h)

            # Рисование частей тела
            # рисуем руки и ноги поверх туловища
            for part, joints in [('left_arm', LEFT_ARM), ('right_arm', RIGHT_ARM),
                                 ('left_leg', LEFT_LEG), ('right_leg', RIGHT_LEG)]:
                for i in range(len(joints) - 1):
                    p1 = get_point(joints[i])
                    p2 = get_point(joints[i + 1])
                    cv2.line(silhouette, p1, p2, COLORS[part], thickness=5)

             # Рисуем торс с полупрозрачной синей заливкой и синим контуром
            torso_points = np.array([get_point(p) for p in TORSO], dtype=np.int32)

            # # Создаём прозрачную заливку
            # overlay = silhouette.copy()
            # transparent_color = (255, 0, 0)  # синий (BGR)
            # alpha = 0.5  # степень прозрачности

            # cv2.fillPoly(overlay, [torso_points], transparent_color)
            # cv2.addWeighted(overlay, alpha, silhouette, 1 - alpha, 0, dst=silhouette)

            # Добавляем чёткий синий контур
            cv2.polylines(silhouette, [torso_points], isClosed=True, color=(255, 0, 0), thickness=3)

            # Рисуем точки суставов
            for lm in mp_pose.PoseLandmark:
                x, y = get_point(lm)
                cv2.circle(silhouette, (x, y), 7, (255, 255, 255), -1)

        # 5. Запись кадра
        out.write(silhouette)
        # cv2_imshow(silhouette)

    cap.release()
    out.release()

In [ ]:
# применяем функцию на всем наборе данных
tqdm.pandas()
df_elements_orig[1000:2000].progress_apply(lambda x: processing_video(x.BD_path, x.BD_silhouette_path), axis=1)

100%|██████████| 1000/1000 [3:52:29<00:00, 13.95s/it]


,0
1000,None
1001,None
1002,None
1003,None
1004,None
...,...
1995,None
1996,None
1997,None
1998,None


In [ ]:
# применяем функцию на всем наборе данных
tqdm.pandas()
df_elements_orig.progress_apply(lambda x: processing_video(x.BD_path, x.BD_silhouette_path), axis=1)

 23%|██▎       | 714/3135 [2:00:08<8:17:46, 12.34s/it]

## Flip

Отзеркаливаем все обработанные видео и сохраняем в целевую папку к остальным видеозаписям

In [ ]:
df_elements_orig['BD_silhouette_path'].head(4)   # пути до обработанных видео

,BD_silhouette_path
0,/content/drive/MyDrive/диплом/Video silhouettes/поворот пассе — сделано в Clipchamp_0.mp4
1,/content/drive/MyDrive/диплом/Video silhouettes/Диана Чугунихина поворт пассе — сделано в Clipchamp_1.mp4
2,/content/drive/MyDrive/диплом/Video silhouettes/Диана Чугунихина поворт пассе — сделано в Clipchamp (1)_2.mp4
3,/content/drive/MyDrive/диплом/Video silhouettes/Диана Чугунихина поворт пассе — сделано в Clipchamp (2)_3.mp4


In [ ]:
# к уже обработанным видео будет добавляться flip
df_elements_orig['BD_silhouette_path'][0].split('/')[-1][:-4] + '_flip.mp4'

'поворот пассе — сделано в Clipchamp_0_flip.mp4'

In [ ]:
df_elements_orig['BD_silhouette_path_flip'] = df_elements_orig['BD_silhouette_path'].str.split('/').apply(lambda x: '/'.join(x[:-1]) + '/' + x[-1][:-4] + '_flip.mp4')
df_elements_orig.reset_index(drop = True, inplace = True)
df_elements_orig[['BD_silhouette_path','BD_silhouette_path_flip']].head(4)

<ipython-input-80-ff446718caaf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elements_orig['BD_silhouette_path_flip'] = df_elements_orig['BD_silhouette_path'].str.split('/').apply(lambda x: '/'.join(x[:-1]) + '/' + x[-1][:-4] + '_flip.mp4')


,BD_silhouette_path,BD_silhouette_path_flip
0,/content/drive/MyDrive/диплом/Video silhouettes new/поворот пассе — сделано в Clipchamp_0.mp4,/content/drive/MyDrive/диплом/Video silhouettes new/поворот пассе — сделано в Clipchamp_0_flip.mp4
1,/content/drive/MyDrive/диплом/Video silhouettes new/Диана Чугунихина поворт пассе — сделано в Clipchamp_1.mp4,/content/drive/MyDrive/диплом/Video silhouettes new/Диана Чугунихина поворт пассе — сделано в Clipchamp_1_flip.mp4
2,/content/drive/MyDrive/диплом/Video silhouettes new/Диана Чугунихина поворт пассе — сделано в Clipchamp (1)_2.mp4,/content/drive/MyDrive/диплом/Video silhouettes new/Диана Чугунихина поворт пассе — сделано в Clipchamp (1)_2_flip.mp4
3,/content/drive/MyDrive/диплом/Video silhouettes new/Диана Чугунихина поворт пассе — сделано в Clipchamp (2)_3.mp4,/content/drive/MyDrive/диплом/Video silhouettes new/Диана Чугунихина поворт пассе — сделано в Clipchamp (2)_3_flip.mp4


In [ ]:
# открываем каждое обработанное видео, отзеркаливаем и сохраняем
def flip_video(input_video_path, output_video_path):

    # открываем видео
    cap = cv2.VideoCapture(input_video_path)   # открываем видео
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # ширина
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))   # высота

    fps = cap.get(cv2.CAP_PROP_FPS)  # частота кадров в 1 секунде

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Основной цикл обработки видео
    while cap.isOpened():  # открываем видео
        ret, frame = cap.read()  # читаем кадры
        if not ret:  # кадры закончились
            break
        frame = cv2.flip(frame, 1)

      # записываем видео
        out.write(frame)
        # cv2_imshow(silhouette)

    cap.release()
    out.release()


In [ ]:
tqdm.pandas()
df_elements_orig.progress_apply(lambda x: flip_video(x.BD_silhouette_path, x.BD_silhouette_path_flip), axis = 1)

100%|██████████| 3135/3135 [1:49:31<00:00,  2.10s/it]


,0
0,None
1,None
2,None
3,None
4,None
...,...
3130,None
3131,None
3132,None
3133,None
